In [2]:
import pandas as pd
import numpy as np

In [3]:
df_meta_data = pd.read_csv('metadata-users.csv')
df_meta_data.head()

User ID  GENDER   AGE                             WORKING    REGION  \
0    36927  Female  60.0                               Other     South   
1     3566  Female  36.0  Full-time housewife / househusband     South   
2    20054  Female  52.0           Employed 30+ hours a week  Midlands   
3    41749  Female  40.0        Employed 8-29 hours per week     South   
4    23108  Female  16.0                   Full-time student     North   

  Hours Listening to Music  
0                   1 hour  
1                   1 hour  
2                   1 hour  
3                  2 hours  
4                  3 hours

In [4]:
df_test = pd.read_csv('test.csv')
df_test.head()

User  Track
0   466      0
1  1173      0
2   566      1
3    49      3
4   772      3

In [5]:
df_train = pd.read_csv('train.csv')
df_train.head()

User  Track  Rating
0     1      0      27
1     4      0      10
2     8      0      30
3    10      0      30
4    11      0      14

In [9]:
ratings = df_train.pivot(index = 'User', columns ='Track', values = 'Rating').fillna(0)
ratings.head()

Track   0     1     2     3    4    5    6    7    8    9   ...   174  175  \
User                                                        ...              
0       0.0  32.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
1      27.0   0.0   0.0  49.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
3       0.0  28.0  30.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
4      10.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
5       0.0   0.0  29.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   

Track  176  177  178  179  180  181  182  183  
User                                           
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 184 columns]

In [13]:
sparsity = float(len(ratings.values.nonzero()[0])) 
sparsity /= (ratings.shape[0] * ratings.shape[1]) 
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 1.71%


In [26]:
R = ratings.as_matrix()
R.shape

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(47856, 184)

In [27]:
class MF():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.
        
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [28]:
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in subtract
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in add
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in double_scalars


Iteration: 10 ; error = nan
Iteration: 20 ; error = nan

P x Q:
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]

Global bias:
36.470711463498716

User bias:
[nan nan nan ... nan nan nan]

Item bias:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan


In [29]:
mf.mse()

nan